# User parameteres

In [1]:
TRIG_XDMF_PATH = "trig/plume_trig.xdmf"         # cell mesh (Grid)
FACETS_XDMF_PATH = "trig/plume_trig_mt.xdmf"    # facet tags (Grid)
OUTPUT_XDMF_PATH_WIRE = "trig/wire_temperature.xdmf" # output file for wire temperature
OUTPUT_XDMF_PATH_TEMP = "trig/temperature.xdmf" # output file for wire temperature
OUTPUT_XDMF_PATH_AIR_T = "trig/air_temperature.xdmf" # output file for wire temperature
OUTPUT_XDMF_PATH_AIR_PV = "trig/air_pressure_velocity.xdmf" # output file for wire temperature
OUTPUT_XDMF_PATH_AIR_PVT = "trig/air_pressure_velocity_temperature.xdmf" # output file for wire temperature
MESH_NAME = "Grid"                              # XDMF mesh name used when writing
GEOM_FILE = "geom.geo"                          # Gmsh geometry file
MSH_FILE = "trig/plume_trig.msh"                # Gmsh mesh file
ELEM = "triangle"                               # element type

WIRE_TAG = 10                                   # wire's cell tag id
AIR_TAG = 11                                    # air's cell tag id
SYMMETRY_AIR_TAG = 100                          # wire/air interface facet tag id 
OUTER_AIR_TAG = 101                             # wire/air interface facet tag id 
INTERFACE_TAG = 102                             # wire/air interface facet tag id 
PRINT_TAG_SUMMARY = True                        # print summary after reading mesh
UNIFORM_Q = True                                # use uniform heat generation

# Material and model params (wire)
k_wire = 16.0                                   # W/(m·K)  thermal conductivity
h_conv = 15.0                                   # W/(m²·K) effective convection at wire boundary
q_wire = 9.75                                       # W/m³ heat generation

k_air = 0.0257                                  # W/(m·K)  thermal conductivity
rho_air = 1.1614                                # kg/m³   density
q_air = 0.0                                     # W/m³    heat generation
mu_air = 1.85e-5                                # kg/(m·s) dynamic viscosity
cp_air = 1007.0                                 # J/(kg·K) specific heat capacity
beta_air = 3.4e-3                               # 1/K volumetric thermal expansion coefficient

# Heating options
I = 1.0                                         # A
sigma_e = 1.0e6                                 # S/m (example)
D_wire = 0.075                                  # m
T_ambient = 292.95                              # K (19.8°C)
g = 9.81                                    # m/s² gravitational acceleration

# Stabilization / iteration
max_it = 20
rtol = 1e-6


# Imports

In [ ]:
# imports
import subprocess
import dolfinx
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, assemble_scalar,
                         form, locate_dofs_geometrical, locate_dofs_topological,
                         FunctionSpace, dirichletbc, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import create_unit_square, locate_entities, create_submesh,meshtags,locate_entities_boundary
from dolfinx.plot import vtk_mesh
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells

from ufl import (SpatialCoordinate, TestFunction, TrialFunction, dx, grad, inner, sym, div, ds, avg, jump, FacetNormal, 
                 Identity, cross, dot, nabla_div, nabla_grad, VectorConstant, finiteelement)

from ufl import (FacetNormal, Identity, Measure, TestFunction, TrialFunction,
                 as_vector, div, dot, ds, dx, inner, lhs, grad, nabla_grad, rhs, sym, system)

from basix import CellType, create_element
from basix.ufl import element, mixed_element

import ufl
import mpi4py.MPI as MPI
import gmsh
import pyvista
import meshio
import numpy as np
import matplotlib.pyplot as plt

import numpy.typing as npt
import petsc4py.PETSc as PETSc
from basix.ufl import element, mixed_element
from dolfinx import default_real_type, la
from dolfinx.fem import (
    Constant,
    Function,
    bcs_by_block,
    dirichletbc,
    extract_function_spaces,
    form,
    functionspace,
    locate_dofs_topological,
)
from dolfinx.fem.petsc import (
    LinearProblem,
    apply_lifting,
    assemble_matrix,
    assemble_vector,
    create_vector,
    set_bc,
)
from dolfinx.io import XDMFFile
# from dolfinx.la.petsc import create_vector_wrap
from dolfinx.mesh import CellType, create_rectangle, locate_entities_boundary

from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver


# Helper functions

In [3]:
# helper functions

def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:, :2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read": [cell_data.astype(np.int32)]})
    return out_mesh

def save_experiment(OUTPUT_XDMF_PATH, mesh, sol:list):
    with XDMFFile(MPI.COMM_WORLD, OUTPUT_XDMF_PATH, "w") as xdmf:
        xdmf.write_mesh(mesh)
        for s in sol:
            xdmf.write_function(s)

    if MPI.COMM_WORLD.rank == 0:
        print("Solved heat equation on wire submesh. Output: ", OUTPUT_XDMF_PATH)

def generate_mesh(GEOM_FILE, MSH_FILE, TRIG_XDMF_PATH, FACETS_XDMF_PATH, ELEM, PRUNE_Z=True):
    subprocess.run(f"gmsh {GEOM_FILE}", shell=True, check=True)

    # Create and save one file for the mesh, and one file for the facets
    msh = meshio.read(MSH_FILE)
    element_mesh = create_mesh(msh, ELEM, prune_z=PRUNE_Z)
    line_mesh = create_mesh(msh, "line", prune_z=PRUNE_Z)
    meshio.write(TRIG_XDMF_PATH, element_mesh)
    meshio.write(FACETS_XDMF_PATH, line_mesh)

def read_mesh(TRIG_XDMF_PATH, FACETS_XDMF_PATH, MESH_NAME, PRINT_TAG_SUMMARY=True):
    with XDMFFile(MPI.COMM_WORLD, TRIG_XDMF_PATH, "r") as xdmf:
        mesh = xdmf.read_mesh(name=MESH_NAME)
        ct = xdmf.read_meshtags(mesh, name=MESH_NAME)
        
    mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim - 1)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)

    with XDMFFile(MPI.COMM_WORLD, FACETS_XDMF_PATH, "r") as xdmf:
        ft = xdmf.read_meshtags(mesh, name=MESH_NAME)

    if PRINT_TAG_SUMMARY and MPI.COMM_WORLD.rank == 0:
        print("Cell tags: ", np.unique(ct.values))
        print("Facet tags: ", np.unique(ft.values))

    return mesh, ct, ft


Build submesh for the wire (cells with tag == WIRE_TAG)

Solve the volumetric heating just for the wire domain

In [4]:
# identify wire cells
def extract_submesh(mesh, ct, ft, SUBDOMAIN_TAG):
    subdomain_cells = np.flatnonzero(ct.values == SUBDOMAIN_TAG)
    subdomain_mesh, subdomain_cell_map, subdomain_vertex_map, _ = create_submesh(mesh, mesh.topology.dim, subdomain_cells)
    subdomain_ft, subdomain_ft_map = transfer_meshtags_to_submesh(mesh, ft, subdomain_mesh, subdomain_vertex_map, subdomain_cell_map)
    subdomain_ds = ufl.Measure(integral_type="ds", domain=subdomain_mesh, subdomain_data=subdomain_ft)
    subdomain_dx = ufl.Measure(integral_type="dx", domain=subdomain_mesh, subdomain_data=ct)
    
    subdomain_mesh.topology.create_connectivity(subdomain_mesh.topology.dim, subdomain_mesh.topology.dim - 1)
    subdomain_mesh.topology.create_connectivity(subdomain_mesh.topology.dim - 1, subdomain_mesh.topology.dim)

    return subdomain_mesh, subdomain_ds, subdomain_dx, subdomain_cell_map, subdomain_ft


def transfer_meshtags_to_submesh(
    mesh: dolfinx.mesh.Mesh,
    entity_tag: dolfinx.mesh.MeshTags,
    submesh: dolfinx.mesh.Mesh,
    sub_vertex_to_parent: npt.NDArray[np.int32],
    sub_cell_to_parent: npt.NDArray[np.int32]
    )-> tuple[dolfinx.mesh.MeshTags, npt.NDArray[np.int32]]:
    """
    Transfer a meshtag from a parent mesh to a sub-mesh.

    Args:
        mesh: Mesh containing the meshtags
        entity_tag: The meshtags object to transfer
        submesh: The submesh to transfer the `entity_tag` to
        sub_to_vertex_map: Map from each vertex in `submesh` to the corresponding
            vertex in the `mesh`
        sub_cell_to_parent: Map from each cell in the `submesh` to the corresponding
            entity in the `mesh`
    Returns:
        The entity tag defined on the submesh, and a map from the entities in the
        `submesh` to the entities in the `mesh`.

    """

    tdim = mesh.topology.dim
    cell_imap = mesh.topology.index_map(tdim)
    num_cells = cell_imap.size_local + cell_imap.num_ghosts
    mesh_to_submesh = np.full(num_cells, -1)
    mesh_to_submesh[sub_cell_to_parent] = np.arange(
        len(sub_cell_to_parent), dtype=np.int32
    )
    sub_vertex_to_parent = np.asarray(sub_vertex_to_parent)

    submesh.topology.create_connectivity(entity_tag.dim, 0)

    num_child_entities = (
        submesh.topology.index_map(entity_tag.dim).size_local
        + submesh.topology.index_map(entity_tag.dim).num_ghosts
    )
    submesh.topology.create_connectivity(submesh.topology.dim, entity_tag.dim)

    c_c_to_e = submesh.topology.connectivity(submesh.topology.dim, entity_tag.dim)
    c_e_to_v = submesh.topology.connectivity(entity_tag.dim, 0)

    child_markers = np.full(num_child_entities, 0, dtype=np.int32)

    mesh.topology.create_connectivity(entity_tag.dim, 0)
    mesh.topology.create_connectivity(entity_tag.dim, mesh.topology.dim)
    p_f_to_v = mesh.topology.connectivity(entity_tag.dim, 0)
    p_f_to_c = mesh.topology.connectivity(entity_tag.dim, mesh.topology.dim)
    sub_to_parent_entity_map = np.full(num_child_entities, -1, dtype=np.int32)
    for facet, value in zip(entity_tag.indices, entity_tag.values):
        facet_found = False
        for cell in p_f_to_c.links(facet):
            if facet_found:
                break
            if (child_cell := mesh_to_submesh[cell]) != -1:
                for child_facet in c_c_to_e.links(child_cell):
                    child_vertices = c_e_to_v.links(child_facet)
                    child_vertices_as_parent = sub_vertex_to_parent[child_vertices]
                    is_facet = np.isin(
                        child_vertices_as_parent, p_f_to_v.links(facet)
                    ).all()
                    if is_facet:
                        child_markers[child_facet] = value
                        facet_found = True
                        sub_to_parent_entity_map[child_facet] = facet
    tags = dolfinx.mesh.meshtags(
        submesh,
        entity_tag.dim,
        np.arange(num_child_entities, dtype=np.int32),
        child_markers,
    )
    tags.name = entity_tag.name
    return tags, sub_to_parent_entity_map


# Problem setup

Initial setup

In [ ]:
generate_mesh(GEOM_FILE, MSH_FILE, TRIG_XDMF_PATH, FACETS_XDMF_PATH, ELEM)
mesh, ct, ft = read_mesh(TRIG_XDMF_PATH, FACETS_XDMF_PATH, MESH_NAME, PRINT_TAG_SUMMARY)
wire_mesh, wire_ds, wire_dx, wire_cells, wire_mt = extract_submesh(mesh, ct, ft, WIRE_TAG)
air_mesh, air_ds, air_dx, air_cells, air_mt = extract_submesh(mesh, ct, ft, AIR_TAG)

# Measures on full mesh
full_dx = dx(domain=mesh, subdomain_data=ct)
full_ds = ds(domain=mesh, subdomain_data=ft)


Cell tags:  [10 11]
Facet tags:  [100 101 102 103]


Variational problem for the wire (entire temperature domain)

In [6]:
# STEP A: Conduction on full mesh (piecewise k, q)
# --------------------
V_T_full = functionspace(mesh, ("Lagrange", 1))
T = TrialFunction(V_T_full)
v = TestFunction(V_T_full)

# DG0 fields for k and q
V0 = functionspace(mesh, ("DG", 0))
k_func = Function(V0)
q_func = Function(V0)

# Fill cellwise values
k_vals = np.full_like(ct.values, k_air, dtype=np.double)
k_vals[ct.values == WIRE_TAG] = k_wire
q_vals = np.full_like(ct.values, q_air, dtype=np.double)
q_vals[ct.values == WIRE_TAG] = q_wire

k_func.x.array[:] = k_vals[ct.indices]
q_func.x.array[:] = q_vals[ct.indices]

T_inf = Constant(mesh, T_ambient)
h = Constant(mesh, h_conv)

# Weak form: -(∇·k∇T) = q, with Robin on outer air boundary (tag 101)
a_T = k_func*inner(grad(T), grad(v))*dx + h*T*v*full_ds(OUTER_AIR_TAG)
L_T = q_func*v*dx + h*T_inf*v*full_ds(OUTER_AIR_TAG)

# (Optional) single-point anchor if you set h_out=0
bcs_T = []

problem_T = LinearProblem(a_T, L_T, bcs=bcs_T,petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
T_full = problem_T.solve()
T_full.name = "T_conduction_full"

save_experiment(OUTPUT_XDMF_PATH_TEMP, mesh, [T_full])


Solved heat equation on wire submesh. Output:  trig/temperature.xdmf


!! Depriciated !!

Variational problem for the wire (wire temperature subdomaindomain)

In [ ]:
# --------------------
# Depreciated, solution on wire submesh only
# --------------------

V = functionspace(wire_mesh, ("CG", 1))
T = TrialFunction(V)
v = TestFunction(V)

# Cell-wise values
k = Constant(wire_mesh, k_wire)
q = Constant(wire_mesh, q_wire)
h = Constant(wire_mesh, h_conv)
Tinf = Constant(wire_mesh, T_ambient)

a_T = k * inner(grad(T), grad(v)) * dx + h * T * v * wire_ds(INTERFACE_TAG)
L_T = q * v * dx + h * Tinf * v * wire_ds(INTERFACE_TAG)
bcs = []

wire_problem = LinearProblem(a_T, L_T, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

wire_sol = wire_problem.solve()
wire_sol.name = "Temperature"

T_wire = wire_sol


Coupling of the 2 domains

In [7]:
# Extract air submesh and map facets
# --------------------
air_cells = np.flatnonzero(ct.values == AIR_TAG)
air_mesh, air_cell_map, _, _ = create_submesh(mesh, mesh.topology.dim, air_cells)

# Map parent facet tags to submesh facets using midpoint matching (local)
parent_fdim = mesh.topology.dim
sub_fdim = air_mesh.topology.dim
mesh.topology.create_connectivity(parent_fdim, parent_fdim-1)
mesh.topology.create_connectivity(parent_fdim-1, parent_fdim)
air_mesh.topology.create_connectivity(sub_fdim, sub_fdim-1)
air_mesh.topology.create_connectivity(sub_fdim-1, sub_fdim)
air_mesh.topology.create_connectivity(sub_fdim, sub_fdim)

pf_inds = []
pf_vals = []
conn_pf_cells = mesh.topology.connectivity(parent_fdim-1, mesh.topology.dim)
for pf, val in zip(ft.indices, ft.values):
    cells = conn_pf_cells.links(pf)
    if np.any(np.isin(cells, air_cells)):
        pf_inds.append(pf)
        pf_vals.append(val)
pf_inds = np.array(pf_inds, dtype=np.int32)
pf_vals = np.array(pf_vals, dtype=np.int32)

coords_parent = mesh.geometry.x
pf_to_verts = mesh.topology.connectivity(parent_fdim-1, 0)
parent_mid = np.zeros((pf_inds.size, coords_parent.shape[1]))
for i, pf in enumerate(pf_inds):
    verts = pf_to_verts.links(pf)
    parent_mid[i,:] = coords_parent[verts,:].mean(axis=0)

sub_to_verts = air_mesh.topology.connectivity(sub_fdim-1, 0)
local_sub_count = air_mesh.topology.index_map(sub_fdim).size_local
sub_mid = np.zeros((local_sub_count, air_mesh.geometry.x.shape[1]))
for f in range(local_sub_count):
    verts = sub_to_verts.links(f)
    sub_mid[f,:] = air_mesh.geometry.x[verts,:].mean(axis=0)

matches = []
vals = []
tol = 1e-8
for i, pm in enumerate(parent_mid):
    d = np.linalg.norm(sub_mid - pm, axis=1)
    j = int(np.argmin(d))
    if d[j] < tol:
        matches.append(j)
        vals.append(pf_vals[i])

if len(matches) == 0:
    air_mt = meshtags(air_mesh, sub_fdim, np.array([], dtype=np.int32), np.array([], dtype=np.int32))
else:
    air_mt = meshtags(air_mesh, sub_fdim, np.asarray(matches, dtype=np.int32), np.asarray(vals, dtype=np.int32))

air_ds = ds(domain=air_mesh, subdomain_data=air_mt)
air_dx = dx(domain=air_mesh)

# --------------------
# Build Dirichlet T on interface from T_full
# --------------------
V_T_air = functionspace(air_mesh, ("Lagrange", 1))
T_air_fun = Function(V_T_air)

if air_mt.values.size:
    interface_facets = air_mt.indices[air_mt.values == INTERFACE_TAG]
else:
    interface_facets = np.array([], dtype=np.int32)


interface_dofs = locate_dofs_topological(V_T_air, sub_fdim, interface_facets)
coords_air = V_T_air.tabulate_dof_coordinates()
xb = coords_air[interface_dofs]

bbt = bb_tree(mesh, mesh.topology.dim)
Tvals = np.zeros((xb.shape[0], 1), dtype=np.double)
for i, x in enumerate(xb):      
    coll = compute_collisions_points(bbt, x)
    cells = compute_colliding_cells(mesh, coll, x)
    cid = int(cells.array[0])
    Tvals[i,0] = T_full.eval(x, cid)

# fill T_gamma
T_gamma = Function(V_T_air)
T_gamma.x.array[:] = T_ambient
if interface_dofs.size > 0:
    T_gamma.x.array[interface_dofs] = Tvals.ravel()

bc_T_interface = dirichletbc(T_gamma, interface_dofs)


Variational problem for the air

-----------

In [ ]:
# started attempt, not finished, still in progress -> Not Working

TH = mixed_element([P2, P1])
W = functionspace(air_mesh, TH)

# No slip boundary condition
W0 = W.sub(0)
Q, _ = W0.collapse()
noslip = Function(Q)
facets = locate_entities_boundary(air_mesh, 1, noslip_boundary)
dofs = locate_dofs_topological((W0, Q), 1, facets)
bc0 = dirichletbc(noslip, dofs, W0)

# Driving velocity condition u = (1, 0) on top boundary (y = 1)
lid_velocity = Function(Q)
lid_velocity.interpolate(lid_velocity_expression)
facets = locate_entities_boundary(air_mesh, 1, lid)
dofs = locate_dofs_topological((W0, Q), 1, facets)
bc1 = dirichletbc(lid_velocity, dofs, W0)

# Collect Dirichlet boundary conditions
bcs = [bc0, bc1]

# Define variational problem
(u, p) = ufl.TrialFunctions(W)
(v, q) = ufl.TestFunctions(W)
f = Function(Q)
a = form((inner(grad(u), grad(v)) + inner(p, div(v)) + inner(div(u), q)) * dx)
L = form(inner(f, v) * dx)

# Assemble LHS matrix and RHS vector
A = fem.petsc.assemble_matrix(a, bcs=bcs)
A.assemble()
b = fem.petsc.assemble_vector(L)

fem.petsc.apply_lifting(b, [a], bcs=[bcs])
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Set Dirichlet boundary condition values in the RHS
for bc in bcs:
    bc.set(b)

# Create and configure solver
ksp = PETSc.KSP().create(air_mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")

# Configure MUMPS to handle pressure nullspace
pc = ksp.getPC()
pc.setType("lu")
# pc.setFactorSolverType("mumps")
# pc.setFactorSetUpSolverType()
# pc.getFactorMatrix().setMumpsIcntl(icntl=24, ival=1)
# pc.getFactorMatrix().setMumpsIcntl(icntl=25, ival=0)

pc.setFactorSolverType("superlu_dist")

# Compute the solution
U = Function(W)
try:
    ksp.solve(b, U.x.petsc_vec)
except PETSc.Error as e:
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

# Split the mixed solution and collapse
u, p = U.sub(0).collapse(), U.sub(1).collapse()


------------

In [ ]:
# Fenicsx community question based. -> Not Working

# Time-stepping parameters
dt = 0.005
T_end = 0.5
t = 0.0
num_steps = int(T_end / dt)

# Define material properties
rho_0 = 1.0  # Density
mu = 1.0  # Dynamic viscosity
K = 1.0  # Thermal conductivity
C_p = 1.0  # Heat capacity
alpha = K / (rho_0 * C_p)  # Thermal diffusivity
Ra = 1.e3  # Rayleigh number

def corner_point(x):
    return np.logical_and(np.isclose(x[0], 40.0), np.isclose(x[1], 100.0))


In [ ]:
P2 = element("Lagrange", mesh.basix_cell(), degree=2, shape=(mesh.geometry.dim,), dtype=default_real_type)
P1 = element("Lagrange", mesh.basix_cell(), degree=1, dtype=default_real_type)
V, Q, R = functionspace(mesh, P2), functionspace(mesh, P1), functionspace(mesh, P1)

TH = mixed_element([P2, P1, P1])
W = functionspace(mesh, TH)

# Define trial and test functions
w = Function(W)
w0 = Function(W)  # Previous solution
(v, p, T) = ufl.split(w)  # Trial functions
(v_test, p_test, T_test) = ufl.TestFunctions(W)  # Test functions

w0.x.array[:] = 0.0


# Collapse the subspaces
V_vel = W.sub(0).collapse()[0]
p_space = W.sub(1).collapse()[0]
T_space = W.sub(2).collapse()[0]

# Define initialization functions
def T_init(x):
    values = np.full(x.shape[1], T_ambient, dtype=np.double)
    return values

def V_init(x):
    return np.zeros((mesh.geometry.dim, x.shape[1]))

def P_init(x):
    return np.zeros((1, x.shape[1]))


# Assign the interpolated values back to the mixed function space
w0.sub(0).interpolate(V_init)
w0.sub(1).interpolate(P_init)
w0.sub(2).interpolate(T_init)
# w0.sub(2).x.array[:] = T_full.x.array[:]

(v0, p0, T0) = ufl.split(w0)

# Collapse the subspaces
V_vel = W.sub(0).collapse()[0]
T_space = W.sub(2).collapse()[0]
p_space = W.sub(1).collapse()[0]


In [ ]:
# collect facets for Dirichlet u=0
facets_noslip = []
if ft.values.size:
    facets_noslip.extend(ft.indices[ft.values == OUTER_AIR_TAG].tolist())
    facets_noslip.extend(ft.indices[ft.values == SYMMETRY_AIR_TAG].tolist())
    facets_noslip.extend(ft.indices[ft.values == INTERFACE_TAG].tolist())
facets_noslip = np.array(facets_noslip, dtype=np.int32)

facets_interface = []
if ft.values.size:
    facets_interface.extend(ft.indices[ft.values == INTERFACE_TAG].tolist())
facets_interface = np.array(facets_interface, dtype=np.int32)

facets_outer = []
if ft.values.size:
    facets_outer.extend(ft.indices[ft.values == OUTER_AIR_TAG].tolist())
facets_outer = np.array(facets_outer, dtype=np.int32)

corner_facets = locate_entities_boundary(mesh, mesh.topology.dim - 1, corner_point)

# DOFs
# velocity
boundary = locate_dofs_topological(W.sub(0), mesh.topology.dim - 1, facets_noslip)
# temperature
interface = locate_dofs_topological(W.sub(2), mesh.topology.dim - 1, facets_interface)
outer = locate_dofs_topological(W.sub(2), mesh.topology.dim - 1, facets_outer)
# pressure pin
corner_dofs = locate_dofs_geometrical(p_space, corner_point) #pointwise 

u_noslip_func = Function(V_vel)
u_noslip_func.x.array[:] = 0.0
T_func = Function(T_space)
T_func.x.array[:] = T_full.x.array[:]
p_corner_func = Function(p_space)
p_corner_func.interpolate(lambda x: np.zeros((1, x.shape[1])))

# Define boundary conditions for velocity (no-slip condition)
u_noslip = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)
bc_u = dirichletbc(u_noslip_func, boundary)#, W.sub(0))
bc_T = dirichletbc(T_func, interface)#, W.sub(2))
bc_p = dirichletbc(p_corner_func, corner_dofs)#, W.sub(1))

bcs = [bc_u, bc_T, bc_p]

if np.any(np.isnan(w.x.array)):
    raise RuntimeError(f"NaN detected")



In [ ]:
# Define buoyancy force
f_B = as_vector([0, Ra * T * C_p * mu / K])

v_t = (v - v0) / dt
T_t = (T - T0) / dt

# Define variational forms
dx = dx(metadata={"quadrature_degree": 3 * 2})

F_mass = -div(v) * p_test * dx

F_momentum = (rho_0 * dot(v_t, v_test) * dx
              + rho_0 * dot(dot(grad(v), v), v_test) * dx
              + 2 * mu * inner(sym(grad(v)), sym(grad(v_test))) * dx
              - p * div(v_test) * dx
              - rho_0 * dot(f_B, v_test) * dx)

F_energy = (dot(T_t, T_test) * dx
            - dot(v * T, grad(T_test)) * dx
            + alpha * dot(grad(T), grad(T_test)) * dx)

# Combine all forms
F = F_mass + F_momentum + F_energy


In [ ]:
from dolfinx.log import set_log_level, LogLevel
set_log_level(LogLevel.INFO)  # Enables logging to monitor solver progress


In [ ]:
# Create nonlinear problem and solver
problem = NonlinearProblem(F, w, bcs)
solver = NewtonSolver(mesh.comm, problem)
solver.rtol = 1e-2  # tolerance
solver.atol = 1e-2  # absolute tolerance
solver.max_it = 10  # max iterations (default is 10)
solver.convergence_criterion = "residual"
solver.relaxation_parameter = 0.7  # Add relaxation
solver.report = True

# Configure PETSc linear solver options
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
# opts[f"{option_prefix}ksp_type"] = "gmres"
# opts[f"{option_prefix}pc_type"] = "ilu"
opts[f"{option_prefix}ksp_type"] = "preonly"  # Use direct solver
opts[f"{option_prefix}pc_type"] = "lu"  # Use LU factorization
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"  # Set MUMPS as the solver

ksp.setFromOptions()
solver.solve(w)

w.sub(0).name = "Velocity"
w.sub(1).name = "Pressure"
w.sub(2).name = "Temperature"

save_experiment(OUTPUT_XDMF_PATH_AIR_T, mesh, [w.sub(0), w.sub(1), w.sub(2)])


--------------------------------

In [ ]:
# Tutorial from workshop on multi-physics -> Not Working

P2 = element("Lagrange", mesh.basix_cell(), degree=2, shape=(mesh.geometry.dim,), dtype=default_real_type)
P1 = element("Lagrange", mesh.basix_cell(), degree=1, dtype=default_real_type)
V, P, T = functionspace(mesh, P2), functionspace(mesh, P1), functionspace(mesh, P1)

W = ufl.MixedFunctionSpace(T,V,P)

t, u, p = ufl.TrialFunctions(W)
dt, du, dp = ufl.TestFunctions(W)

dx = ufl.Measure("dx", domain=mesh, subdomain_data=ct)
dx_thermal = dx(WIRE_TAG)
dx_fluid = dx(AIR_TAG)
dx_heat = dx(WIRE_TAG)

g = dolfinx.fem.Constant(air_mesh, dolfinx.default_scalar_type((0, 0)))

f_fluid = dolfinx.fem.Constant(air_mesh, dolfinx.default_scalar_type((0, 0)))
f_heat = 10*ufl.sin(5*ufl.pi*x[1]) + x[0]**3

a = ufl.inner(ufl.grad(u), ufl.grad(du)) * dx_fluid
a += ufl.inner(p, ufl.div(du)) * dx_fluid
a += ufl.inner(ufl.div(u), dp) * dx_fluid
a += ufl.inner(k_func*ufl.grad(t), ufl.grad(dt))*dx_heat

x = ufl.SpatialCoordinate(mesh)

L = ufl.inner(f_fluid, du) * dx_fluid
L += ufl.inner(dolfinx.fem.Constant(air_mesh, 0.0), dp) * dx_fluid
L += ufl.inner(f, dt)*dx_thermal

a_blocked = ufl.extract_blocks(a)
L_blocked = ufl.extract_blocks(L)


In [ ]:
tdim = mesh.topology.dim
cell_map = mesh.topology.index_map(tdim)
num_cells_local = cell_map.size_local + cell_map.num_ghosts

mesh_to_heat_entity = np.full(num_cells_local, -1, dtype=np.int32)
mesh_to_heat_entity[wire_cells] = np.arange(len(wire_cells), dtype=np.int32)
mesh_to_stokes_entity = np.full(num_cells_local, -1, dtype=np.int32)
mesh_to_stokes_entity[air_cells] = np.arange(len(air_cells), dtype=np.int32)
entity_maps = {wire_mesh: mesh_to_heat_entity, air_mesh: mesh_to_stokes_entity}
a_blocked_compiled = dolfinx.fem.form(a_blocked, entity_maps=entity_maps)
L_blocked_compiled = dolfinx.fem.form(L_blocked, entity_maps=entity_maps)

T_bndry = dolfinx.fem.Function(T)
T_bndry.x.array[:] = 0
wire_mesh.topology.create_connectivity(wire_mesh.topology.dim-1, wire_mesh.topology.dim)
heat_bc_dofs = dolfinx.fem.locate_dofs_topological(T, wire_mesh.topology.dim-1, wire_mt.find(OUTER_AIR_TAG))
bc_heat = dolfinx.fem.dirichletbc(T_bndry, heat_bc_dofs)

stokes_walls = np.union1d(air_mt.find(OUTER_AIR_TAG), wire_mt.find(INTERFACE_TAG))
dofs_wall = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, stokes_walls)
u_wall = dolfinx.fem.Function(V)
u_wall.x.array[:] = 0
bc_wall = dolfinx.fem.dirichletbc(u_wall, dofs_wall)

bcs = [bc_heat, bc_wall]


In [ ]:
from petsc4py import PETSc
import dolfinx.fem.petsc

A = dolfinx.fem.petsc.create_matrix_block(a_blocked_compiled)
A.zeroEntries()
dolfinx.fem.petsc.assemble_matrix_block(A, a_blocked_compiled, bcs=bcs)
A.assemble()

b = dolfinx.fem.petsc.create_vector_block(L_blocked_compiled)
dolfinx.fem.petsc.assemble_vector_block(b, L_blocked_compiled, a_blocked_compiled, bcs=bcs)
b.ghostUpdate(PETSc.InsertMode.INSERT_VALUES, PETSc.ScatterMode.FORWARD)

ksp = PETSc.KSP().create(mesh.comm)
ksp.setOperators(A)
ksp.setType(PETSc.KSP.Type.PREONLY)
ksp.getPC().setType(PETSc.PC.Type.LU)
ksp.getPC().setFactorSolverType("mumps")

w_blocked = dolfinx.fem.petsc.create_vector_block(L_blocked_compiled)

ksp.solve(b, w_blocked)
assert ksp.getConvergedReason() > 0, "Solve failed"

blocked_maps = [(space.dofmap.index_map, space.dofmap.index_map_bs) for space in W.ufl_sub_spaces()]
local_values = dolfinx.cpp.la.petsc.get_local_vectors(w_blocked, blocked_maps)

Th = dolfinx.fem.Function(T, name="Temperature")
uh = dolfinx.fem.Function(V, name="Velocity")
ph = dolfinx.fem.Function(Q, name="Pressure")
Th.x.array[:] = local_values[0]
uh.x.array[:] = local_values[1]
ph.x.array[:] = local_values[2]

save_experiment(OUTPUT_XDMF_PATH_AIR_P, mesh, [uh, ph, Th])


----------

In [ ]:
# Based on tutorial for pipe flow -> Not Working
# Define variational problem
(u, p) = ufl.TrialFunctions(W)
(v, q) = ufl.TestFunctions(W)
f = Function(Q)
a = form(
    (ufl.inner(ufl.grad(u), ufl.grad(v)) + ufl.inner(p, ufl.div(v)) + ufl.inner(ufl.div(u), q))
    * ufl.dx
)
L = form(ufl.inner(f, v) * ufl.dx)

# Assemble LHS matrix and RHS vector
A = assemble_matrix(a, bcs=bcs)
A.assemble()
b = assemble_vector(L)

bcs1 = bcs_by_block(extract_function_spaces([[a]], 1), bcs)
apply_lifting(b, [a], bcs=bcs1)
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Set Dirichlet boundary condition values in the RHS
for bc in bcs:
    bc.set(b.array_w)

# Create and configure solver
ksp = PETSc.KSP().create(air_mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")

# Configure MUMPS to handle pressure nullspace
pc = ksp.getPC()
pc.setType("lu")
use_superlu = PETSc.IntType == np.int64
if PETSc.Sys().hasExternalPackage("mumps") and not use_superlu:
    pc.setFactorSolverType("mumps")
    pc.setFactorSetUpSolverType()
    pc.getFactorMatrix().setMumpsIcntl(icntl=24, ival=1)
    pc.getFactorMatrix().setMumpsIcntl(icntl=25, ival=0)
else:
    pc.setFactorSolverType("superlu_dist")

# Compute the solution
U = Function(W)
try:
    ksp.solve(b, U.x.petsc_vec)
except PETSc.Error as e:
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

# Split the mixed solution and collapse
u, p = U.sub(0).collapse(), U.sub(1).collapse()

u.name = "Velocity"
p.name = "Pressure"

save_experiment(OUTPUT_XDMF_PATH_AIR_V, air_mesh, u)
save_experiment(OUTPUT_XDMF_PATH_AIR_P, air_mesh, p)


----------

In [ ]:
# Original approach, output generated, code runs without error, but results incorrect

P2 = element("Lagrange", air_mesh.basix_cell(), degree=3, shape=(air_mesh.geometry.dim,), dtype=default_real_type)
P1 = element("Lagrange", air_mesh.basix_cell(), degree=2, dtype=default_real_type)
V, P, T = functionspace(air_mesh, P2), functionspace(air_mesh, P1), functionspace(air_mesh, P1)
# W = ufl.MixedFunctionSpace(T,V,P)

TH = mixed_element([P2, P1])
W = functionspace(air_mesh, TH)
V_u = W.sub(0).collapse()[0]
V_p = W.sub(1).collapse()[0]

u = Function(V_u, name="u")
p = Function(V_p, name="p")

# trial/test
w = TrialFunction(W)
# Actually get separate TrialFunctions/TestFunctions
(u_trial, p_trial) = ufl.split(TrialFunction(W))
(v_test, q_test) = ufl.split(TestFunction(W))

# buoyancy body force using interface-initialized T_gamma extended (approx)
# use T_gamma as initial temperature field on air; it's defined only on mesh
f_b = as_vector((0.0, -rho_air * g * beta_air * (T_gamma - T_ambient)))

# Stokes bilinear and linear forms (mixed)
a_stokes = 2 * mu_air * inner(sym(grad(u_trial)), sym(grad(v_test))) * air_dx - div(v_test) * p_trial * air_dx + q_test * div(u_trial) * air_dx
L_stokes = inner(f_b, v_test) * air_dx

# # collect facets for Dirichlet u=0
facets_noslip = []
if ft.values.size:
    facets_noslip.extend(ft.indices[ft.values == OUTER_AIR_TAG].tolist())
    facets_noslip.extend(ft.indices[ft.values == SYMMETRY_AIR_TAG].tolist())
    facets_noslip.extend(ft.indices[ft.values == INTERFACE_TAG].tolist())
facets_noslip = np.array(facets_noslip, dtype=np.int32)


# Boundary conditions for velocity: no-slip on outer and symmetry and interface
zero_u = Function(V_u)
zero_u.x.array[:] = 0.0
dofs_u_bnd = locate_dofs_topological(V_u, sub_fdim, facets_noslip)
bc_u = dirichletbc(zero_u, dofs_u_bnd)


# Pressure pin at one dof to remove nullspace
x_p = V_p.tabulate_dof_coordinates()
i0 = int(np.argmin(x_p[:,1]))  # pick lowest point
p0fun = Function(V_p)
p0fun.x.array[:] = 0.0
bc_p = dirichletbc(p0fun, dofs_u_bnd)


# Combine BCs
# bcs_u = [bc_u]
# bcs_p = [bc_p]
bcs_mixed = [bc_u, bc_p]

# Solve mixed Stokes using LinearProblem with W as solution Function
W_u = Function(W)
problem_stokes = LinearProblem(a_stokes, L_stokes, bcs=bcs_mixed, u=W_u, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
W_sol = problem_stokes.solve()



: 

In [ ]:
# split
# u_sol, p_sol = ufl.split(W_sol)
u_sol, p_sol = W_sol.sub(0).collapse(), W_sol.sub(1).collapse()
u_sol.x.scatter_forward()
P1_local = element("Lagrange", air_mesh.basix_cell(), 1, shape=(air_mesh.geometry.dim,), dtype=default_real_type)

u_val = Function(functionspace(air_mesh, P1_local), name="Velocity")
u_val.interpolate(u_sol)

print(len(p_sol.x.array), len(u_val.x.array))

p_val = Function(P, name="Pressure")
p_val.interpolate(p_sol)


save_experiment(OUTPUT_XDMF_PATH_AIR_PV, air_mesh, [u_val,p_val])

print("Solved Stokes in air submesh")
# 


# Save experiment

In [ ]:
save_experiment(OUTPUT_XDMF_PATH_WIRE, wire_mesh, wire_sol)
save_experiment(OUTPUT_XDMF_PATH_AIR, air_mesh, W_sol)


# Biot number for an infinitely long heated wire (2D cross-section)

The Biot number $\mathrm{Bi}$ is a dimensionless parameter that characterises the relative importance of heat transfer resistance inside a solid to the convective resistance at its surface:

$$
\mathrm{Bi} = \frac{h, L_c}{k}
$$

where
*	$h$ is the convective heat transfer coefficient $ [W·m^{-2}·K^{-1}] $,
*	$k$ is the thermal conductivity of the solid (wire) $ [W·m^{-1}·K^{-1}$],
*	$L_c$ is a characteristic length [m].

⸻

Characteristic length for an infinitely long cylinder (2D model)

For an infinitely long wire modelled in 2D (circular cross section of radius $r$), a consistent definition of the characteristic length is the volume-to-surface ratio:

$$
L_c = \frac{V}{A_s}.
$$

Taking a unit length in the axial direction ($ 1m $) gives

$$
V = \pi r^2 \cdot 1, \qquad A_s = 2\pi r \cdot 1
$$

and hence

$$
L_c = \frac{\pi r^2}{2\pi r} = \frac{r}{2}.
$$

Thus, for a long cylindrical wire the conventional choice is $L_c = r/2$.
Note: some texts use $r$ as a characteristic length for scaling; the $ V/A_s $ definition is the standard choice that yields $L_c = r/2$ for a cylinder and ensures consistency with other shapes (slab, sphere, etc.).

⸻

Physical meaning and use

Rewriting $\mathrm{Bi}$ as

$$
\mathrm{Bi} = \frac{\text{convective resistance scale}^{-1}}{\text{conductive resistance scale}^{-1}}
= \frac{h L_c}{k},
$$

indicates whether the internal conduction resistance ($\sim k/L_c$) or the convective surface resistance ($\sim h$) dominates:
*	$\mathrm{Bi} \ll 1$: internal conduction is fast compared to surface convection → solid temperature is nearly spatially uniform (lumped capacitance assumption valid).
*	$\mathrm{Bi} \gg 1$: internal conduction is slow compared to surface convection → significant temperature gradients exist inside the solid.

⸻

Lumped-capacitance criterion

A commonly used practical criterion for the lumped capacitance model is

$$
\mathrm{Bi} \le 0.1,
$$

which implies that internal temperature gradients are small and the solid can be treated as having uniform temperature in transient heat-transfer analyses.

⸻

Thermal-resistance perspective

Consider characteristic conductive resistance

$$
R_{\mathrm{cond}} \sim \frac{L_c}{k A_c},
$$

and convective resistance

$$
R_{\mathrm{conv}} \sim \frac{1}{h A_s}.
$$

Using $A_c \sim A_s$ for the chosen unit length and rearranging yields

$$
\frac{R_{\mathrm{cond}}}{R_{\mathrm{conv}}} \sim \frac{1}{\mathrm{Bi}}.
$$

Thus a small Bi corresponds to small conductive resistance relative to convective resistance.

⸻

Practical note. For an air environment $h$ is typically much smaller than for liquids; thus often is found $\mathrm{Bi}$ for thin wires in air to be small (supporting the lumped assumption) unless the wire radius is large or the wire material has very low thermal conductivity. 
When in doubt, compute

$$
\mathrm{Bi} = \frac{h(r/2)}{k}
$$

with actual $h, k, r$ and check the $\mathrm{Bi} \le 0.1$ rule.

⸻


# Material parameters

1) Thermal expansion coefficient $ \beta $
*   appears in bouyancy

2) Dynamic viscosity $ \mu(T) $ and kinematic viscosity $ \nu(T)=\mu/\rho $ — [Pa·s], [m²/s]
*   Use Sutherland’s law or tabulated data for accuracy. Sutherland form:
$$
\mu(T) = \mu_{\text{ref}}\left(\frac{T}{T_{\text{ref}}}\right)^{3/2}\frac{T_{\text{ref}}+S}{T+S}
$$ 
with Sutherland constant S.
*	Provide both μ and ν because momentum eqns often use μ while numerical time stepping/scale analysis uses ν.

3) Thermal conductivity k(T) — [W·m⁻¹·K⁻¹]
*	Used directly in the diffusion term and for computing thermal diffusivity.

4)	Specific heat at constant pressure $ c_p(T)$  and possibly $c_v(T)$ — [J·kg⁻¹·K⁻¹]
*	Use $ c_p(T)$  for energy equation. If compressible flow, may need $ c_v $ and ratio of specific heats $ \gamma(T)=c_p/c_v $ .

5)	Density \rho(T,p) — [kg·m⁻³]
*	For moderate/large temperature differences, use ideal-gas law:
$ \rho(T,p)=\frac{p}{R_{\text{air}} T} $ (with gas constant $ R_{\text{air}}$ ). Use tabulated real-gas data if needed.

6)	Thermal diffusivity $ \alpha(T)=\dfrac{k(T)}{\rho(T)\,c_p(T)} $— [m²/s]
*	Useful to compute dimensionless groups and for CFL-like criteria.

Thermal diffusivity:
$\alpha(T)=\frac{k(T)}{\rho(T)\,c_p(T)}.$

Prandtl number:
$\mathrm{Pr}(T)=\frac{c_p(T)\,\mu(T)}{k(T)} = \frac{\nu(T)}{\alpha(T)}.$

Sutherland law (viscosity example):
$\mu(T)=\mu_\text{ref}\left(\frac{T}{T_\text{ref}}\right)^{3/2}\frac{T_\text{ref}+S}{T+S}.$

Enthalpy from c_p:
$h(T)=h(T_{\text{ref}})+\int_{T_{\text{ref}}}^{T} c_p(T’)\,dT’.$

Ideal-gas density:
$\rho(T,p)=\frac{p}{R_{\text{air}}\,T}.$

Boussinesq buoyancy force (linearised):
$\mathbf{f}_b \approx -\rho_0 \beta(T-T_0)\mathbf{g}.$